In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
import os
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv

f:\medical-chatbot\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
load_dotenv()

True

In [5]:
#extract data from pdf
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [6]:
extracted_data=load_pdf(r"../data/")

In [7]:
#extracted_data

## Creating text chuncks from corpus

In [7]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks= text_split(extracted_data)

In [9]:
len(text_chunks)

7020

## Download embedding model

In [10]:
def download_hugging_face_embedding():
    embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings=download_hugging_face_embedding()

f:\medical-chatbot\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
test_query_result=embeddings.embed_query("Hello world")
print("length : ",len(test_query_result))

length :  384


In [15]:
#test_query_result

## Initializing the Pinecone 

In [16]:
index_name="medical-chatbot"

# creating embeddings for each of the chunk and storing 
docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)

In [17]:
# vectors are stored inside the pinecone database

In [15]:
index_name="medical-chatbot"

docsearch=PineconeVectorStore.from_existing_index(index_name,embeddings)

query="tell me about Naratriptan"

docs=docsearch.similarity_search(query,k=3)

print("Result : ",docs)

Result :  [Document(page_content='(Cafergot), naratriptan (Amerge), sumatriptan (Imitrex),rizatriptan (Maxalt) and zolmitriptan (Zomig). Methy-sergide maleate (Sansert) may be used by patients whoseheadaches are not controlled by other drugs, while somepatients do well on other drugs. For example, combina-tions or ergotamine and caffeine may be very effective.\nThe caffeine acts by constricting blood vessels to relievethe headache . Sometimes, an analgesic such as aceta-\nminophen , caffeine, and a barbiturate which acts as a'), Document(page_content='Typical recommended dosages for adults are given belowfor each type of drug.Ergotamine\nTake at the first sign of a migraine attack. Patients\nwho get warning signals (aura) may take the drug as soonas they know a headache is coming.\nTABLETS. No more than six tablets for any single\nattack.\nNo more than 10 tablets per week.\nSUPPOSITORIES. No more than two suppositories\nfor any single attack.\nNo more than five suppositories per week.\

In [20]:
#print(docs[0].page_content)

In [16]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [17]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [18]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,'temperature':0.8}
                  )

In [19]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [20]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa.invoke({"query": user_input})
    print("Response : ", result["result"])

Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
Response :  Naratriptan is a triptan class drug that works by constricting blood vessels in the brain to relieve migraine pain. It is typically taken as soon as possible after the start of a migraine attack, but not before the aura symptoms have appeared. The recommended dosage for adults is 2.5 mg to 5 mg taken orally, and the maximum dose per week is 10 mg.


KeyboardInterrupt: Interrupted by user